# %% Lagragien for Static Balance

*Source : Dynamics and Control of np.single-Line Kites - Gonzalo Sanchez-Arrlaga*

*Goal : Find the Angle of Attack ($\alpha$) and the Elevation($\beta$) in Regard of Wind Speed*

# %% Import

In [ ]:
import os
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

# %% Data

In [ ]:
# # %% Xfoil Polar 
polar_data = pd.read_csv("polar_SK50VD_modified.csv") # Same airfoil as 10VB
polar = pd.json_normalize({
    "alpha" : polar_data.alpha,
    "CL" : polar_data.CL,
    "CD" : polar_data.CD, 
    "CDp" : polar_data.CDp,
    "CM" : polar_data.CM,
    "Top_Xtr" : polar_data.Top_Xtr,
    "Bot_Xtr" : polar_data.Bot_Xtr,
})

# # %% Air
g = 9.8 
rho = 1.225

# # %% Kite
Surface = 10 # (m2)
mass = 5.5 # (kg)
height = 2.2 # distance from towpoint to profile (m)
corde = 1.99 # (m)
Xt = 0.3* corde # towpoint (m)
Xg = 0.5 * corde  # mass center (m)
delta_deg = 83 # Bridle Angle (°)
r = 2.26 # Bridle Length |QG| (m)

# %% Calculs 
polar["Xcop"] = (0.25 - polar["CM"]/polar["CL"])* corde
polar["Xcp"] = Xg-polar["Xcop"]
polar["CN"] = np.sqrt(polar["CL"]*polar["CL"]+ polar["CD"]*polar["CD"]) 

delta = delta_deg*np.pi/180 #atan(height / (Xg-Xt))
polar["sigma"] = polar["Xcp"] / r 

# %% Main

In [ ]:
# %% input
W0 =5 # Wind speed (m/s)

# %% equation 1 
beta = rho * Surface * W0^2 / (2 * mass * g)

polar["np.cos1"] = np.np.cos(delta - polar["alpha"]*np.pi/180)
polar["np.cos2"] = - beta * (polar["sigma"] - np.np.cos(delta))* polar["CN"]

polar["sum"] = polar["np.cos1"] + polar["np.cos2"]

# %% Alpha
ax_alpha_tot = sns.lineplot(data=df, x="r.CL", y="alpha_tot", hue="delta_camber")
ax_alpha_tot.set_title("alpha_tot(delta_camber) [deg]")
plt.show()

plot(polar["alpha"], polar["sum"], '*')
xlabel('alpha')
ylabel('sum')
title('equation1')
grid on

# %% polar["theta

polar["anp.cos"] = anp.cos(- beta * (polar["sigma"] - np.cos(delta))*polar["CN"])*180/np.pi
polar["theta1"] = delta_deg - polar["anp.cos"]
polar["theta2"] = -(delta_deg + polar["anp.cos"] - 180)
polar["diff1"] = polar["theta1"] - polar["alpha"]
polar["diff2 "]= polar["theta2"] - polar["alpha"]
# %% equation 2 

polar["gamma"] = atan ((beta * polar["CN"] *np.cos(polar["alpha"]*np.pi/180)-1) ./ (beta * polar["CN"] * np.sin(polar["alpha"]*np.pi/180))  ) *180/np.pi

# %% regression de l'élévation en fonction de l'incidence : 

mdl = fitlm(polar["alpha"], polar["gamma"])

figure
plot(polar["alpha"], polar["gamma"], '*')
hold on
plot(polar["alpha"], predict(mdl, polar["alpha"]), 'r')
xlabel('alpha')
ylabel('gamma')
title('regression linéaire')
legend('données', 'ligne de regression')


# Coefficients du modèle
coefficients = mdl.Coefficients.Estimate
intercept = coefficients(1)
slope = coefficients(2)

# R²
r_squared = mdl.Rsquared.Ordinary

# Afficher les résultats
print('Equation: y = #.2f + #.2f*x\n', intercept, slope)
print('R²: #.2f\n', r_squared)